<a href="https://colab.research.google.com/github/yyqbb/DATA201-Group-Project-2022/blob/main/DATA301_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
filename = 'renttherunway_final_data.json.gz'
urllib.request.urlretrieve('https://datarepo.eng.ucsd.edu/mcauley_group/data/renttherunway/renttherunway_final_data.json.gz', filename)

('renttherunway_final_data.json.gz',
 <http.client.HTTPMessage at 0x799589f80280>)

In [ ]:
!pip install ijson

In [ ]:
import urllib.request
import gzip
import shutil
import json

# Download the compressed file
filename = "renttherunway_final_data.json.gz"
url = "https://datarepo.eng.ucsd.edu/mcauley_group/data/renttherunway/renttherunway_final_data.json.gz"
urllib.request.urlretrieve(url, filename)

# Decompress the file
decompressed_filename = "renttherunway_final_data.json"
with gzip.open(filename, "rb") as f_in:
    with open(decompressed_filename, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

print("File decompressed successfully:", decompressed_filename)

def filter_data_and_update(filename):
    updated_entries = []
    with open(filename, 'r') as f:
        for line in f:
            entry = json.loads(line)
            if "body type" in entry and "rating" in entry and "review_text" in entry and "fit" in entry:
                updated_entries.append({
                    "body type": entry["body type"],
                    "rating": entry["rating"],
                    "review_text": entry["review_text"],
                    "fit": entry["fit"]  # Include the "fit" field
                })

    # Write the updated entries back to the file
    with open(filename, 'w') as f:
        for entry in updated_entries:
            json.dump(entry, f)
            f.write('\n')

    print("File updated successfully")

# Update the file
filter_data_and_update(decompressed_filename)


File decompressed successfully: renttherunway_final_data.json
File updated successfully


In [ ]:
!head -10 'renttherunway_final_data.json'
!tail -10 'renttherunway_final_data.json'

{"body type": "hourglass", "rating": "10", "review_text": "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.", "fit": "fit"}
{"body type": "straight & narrow", "rating": "10", "review_text": "I rented this dress for a photo shoot. The theme was \"Hollywood Glam and Big Beautiful Hats\". The dress was very comfortable and easy to move around in. It is definitely on my list to rent again for another formal event. ", "fit": "fit"}
{"body type": "pear", "rating": "8", "review_text": "I rented this for my company's black tie awards banquet.  I liked that this dress was short but was a little fancier with the sequins. (I generally don't care for long dresses.)  I would describe the color as more rose gold than yellow gold.  I have blonde hair and fair skin, and the color was very flattering.  This is a very forgiving dress

In [ ]:
import os

filename = 'renttherunway_final_data.json'

# Check if the file exists
if os.path.exists(filename):
    # Get the size of the file in bytes
    file_size = os.path.getsize(filename)

    if file_size > 0:
        print(f"The file '{filename}' contains data. Size: {file_size} bytes.")
    else:
        print(f"The file '{filename}' is empty.")
else:
    print(f"The file '{filename}' does not exist.")

The file 'renttherunway_final_data.json' contains data. Size: 64925918 bytes.


In [ ]:
import json
import pandas as pd

# Read the JSON file into a DataFrame
df = pd.read_json('renttherunway_final_data.json', lines=True)

# Group by 'body type' and calculate the mean rating for each group
mean_ratings = df.groupby('body type')['rating'].mean().reset_index()

# Sort the DataFrame by mean rating in descending order
mean_ratings_sorted = mean_ratings.sort_values(by='rating', ascending=False)

# Output the mean ratings for each body type
print("Mean Ratings for Each Body Type (Highest to Lowest):")
print(mean_ratings_sorted.to_string(index=False))


Mean Ratings for Each Body Type (Highest to Lowest):
        body type   rating
straight & narrow 9.157338
         athletic 9.122010
           petite 9.079732
        hourglass 9.077783
             pear 9.041399
        full bust 9.022468
            apple 8.928030


In [ ]:
import dask
# from dask import dataframe as ddf
from dask import bag as db

# data_df = ddf.read_json('filter-all-t.jsonl', blocksize="1MB")
data_bag = db.read_text('renttherunway_final_data.json', blocksize="1MB")
print(data_bag)

dask.bag<bag-from-delayed, npartitions=18>


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Select relevant features for clustering
features = df[['size', 'age']]

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Determine the optimal number of clusters using the Elbow Method
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(scaled_features)
    inertia.append(kmeans.inertia_)

# Plot the Elbow Method to visualize the optimal number of clusters
plt.plot(range(1, 11), inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()

# Based on the Elbow Method, choose the optimal number of clusters
k = 3

# Perform k-means clustering
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(scaled_features)

# Add cluster labels to the DataFrame
df['cluster'] = clusters

# Output cluster centers
print("Cluster Centers:")
print(scaler.inverse_transform(kmeans.cluster_centers_))

# Visualize clusters
plt.scatter(df['size'], df['age'], c=clusters, cmap='viridis', alpha=0.5)
plt.xlabel('Size')
plt.ylabel('Age')
plt.title('Cluster Analysis')
plt.show()


NameError: name 'df' is not defined

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

STOP_WORDS = set(stopwords.words('english'))
print(STOP_WORDS)

{'did', "shan't", 'weren', 'all', 'its', 'are', 'which', 'yours', 'too', 'no', "she's", 'to', 'any', "doesn't", 'needn', 'as', 'herself', 'whom', 'couldn', 'up', 'against', 'other', 'each', 'how', 'a', 'they', 'you', "aren't", 'just', 'again', 'wouldn', 'myself', 'only', 'am', 'if', 'will', 'your', 'own', "hasn't", 'those', 'aren', 'can', 'll', "couldn't", "mightn't", "should've", 'themselves', 'my', 'isn', "that'll", 'an', "you've", 'while', 'who', 'is', 'ours', "you're", 'do', 're', 'their', 'not', 'ain', 'hasn', 'nor', 'into', 'ma', 'where', 'what', 'when', 't', "hadn't", 'these', "don't", "haven't", 'but', 'that', 'very', 'by', 'during', 'shan', 'her', "it's", 'until', 'i', "won't", 'were', 'before', 'same', 'me', 'he', "shouldn't", 'himself', 'wasn', 'them', 'both', 'now', "wouldn't", 'between', 'why', 'we', 've', 'yourselves', 'theirs', 'the', 'don', 'itself', 'mustn', 'ourselves', 'be', 'yourself', 'under', 'd', 'about', "needn't", 'out', 's', 'further', 'our', 'more', 'haven', 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import re
import json

def remove_nonletters(word):
    return re.sub(r'[^a-zA-Z]', '', word)

def process_line(line):
    entry = json.loads(line)
    results = []
    body_type = entry['body type']  # Adjusted to 'body type'
    # removes punctuation and converts to lowercase
    review = re.sub(r'[^a-zA-Z ]', '', entry['review_summary'].lower())
    for word in review.split(" "):
        # don't keep any words that are in STOP_WORDS
        if len(word) > 0 and word not in STOP_WORDS:
            results.append((body_type, word))
    return results

reviews_only_valid_words = data_bag.map(process_line).flatten()
body_type_words_df = reviews_only_valid_words.to_dataframe(columns=['body type', 'word'])  # Adjusted to 'body type'
body_type_words_df_with_counts = body_type_words_df.groupby(['body type','word']).size().to_frame('counts').reset_index()  # Adjusted to 'body type'
for body_type in body_type_words_df_with_counts['body type'].unique().compute():  # Adjusted to 'body type'
    print(f"Top ten words for {body_type} body type:")
    print(body_type_words_df_with_counts[body_type_words_df_with_counts['body type'] == body_type].nlargest(10, 'counts').compute())  # Adjusted to 'body type'


Top ten words for apple body type:
    body type         word  counts
97      apple        dress    1998
168     apple        great     596
28      apple    beautiful     506
273     apple      perfect     504
73      apple  compliments     497
68      apple  comfortable     415
221     apple        loved     324
425     apple      wedding     319
64      apple        color     266
440     apple         wore     262
Top ten words for athletic body type:
     body type         word  counts
877   athletic        dress   18621
1098  athletic        great    5413
1504  athletic      perfect    5178
750   athletic  compliments    4651
565   athletic    beautiful    4137
734   athletic  comfortable    3441
1299  athletic        loved    3420
2024  athletic      wedding    3068
726   athletic        color    2846
997   athletic          fit    2740
Top ten words for full bust body type:
      body type         word  counts
2319  full bust        dress    6214
2441  full bust        great    1